# Stage 2: Nash equilibrium

In [1]:
using JuMP, Cbc, NamedArrays
import Combinatorics

In [23]:
# one is enouxgh!! No of games - 1's score = 2's score
elo = [2838, 2822, 2817, 2772, 2771, 2761, 2750, 2747, 2745, 2741]
price = [1000, 900, 850, 700, 690, 650, 600, 580, 570, 560] * 0.001 

team_host = [1 3 5 6]
team_guest = [2 4 7 8]

plan_guest = [2 1 4 3] # fixed plan

N_PLAYERS = 4
N_CHOSEN = 4
N_POOL = 10
;

### Game score function

In [9]:
# the positional advantage and the ratings difference
# higher elo, first move better
# order: 1 or 0 with 1 being first
function get_score_chess_game(player, opponent, order, elo)
    x = elo[player] - elo[opponent] + 35*(-1)^(order + 1)
    return 1/(1 + 10^(-x/400)) 
end
;

### Match score function

In [10]:
function get_match_score(team_player, team_opponent, order, elo)
    sum(get_score_chess_game(team_player[i], team_opponent[i], 
            (i + order + 1) % 2, elo) for i=1:length(team_player))
end
;

### Generate score matrices for all players
Suppose that we have 10 players, choose 4 of them to establish our team, 6 other players become our opponents

### Nash equilibrium

X site

In [12]:
nashModel = Model()

@variable(nashModel, x[1:N_PLAYERS, 1:N_PLAYERS], Bin)
@variable(nashModel, p[1:N_PLAYERS])
@variable(nashModel, q[1:N_PLAYERS])
@variable(nashModel, r[1:N_PLAYERS, 1:N_PLAYERS] <= 0)

@constraint(nashModel, supply[k in 1:N_PLAYERS], 
                sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(nashModel, demand[k in 1:N_PLAYERS], 
                sum(x[i, k] for i=1:N_PLAYERS) == 1)
                        
@constraint(nashModel, mincons[j in 1:N_PLAYERS, k in 1:N_PLAYERS], 
    p[j] + q[k] + r[j, k] <= 
        sum(get_score_chess_game(team_host[i], team_guest[j], k, elo) 
                                * x[i, k] for i=1:N_PLAYERS))
                                    
@objective(nashModel, Max, sum(p) + sum(q) + sum(r))        

solve(nashModel)
                                    
println(getobjectivevalue(nashModel))
xopt = getvalue(x)
solution_x = NamedArray(Int[xopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.1368277781896134
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  0  1
2     │ 0  1  0  0
3     │ 1  0  0  0
4     │ 0  0  1  0
p: [0.472524,0.544223,0.575687,0.5799]
q: [0.0042012,-0.0298361,-0.00987175,0.0]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [13]:
ym = Model()

@variable(ym, y[1:N_PLAYERS, 1:N_PLAYERS], Bin)

@constraint(ym, supply[k in 1:N_PLAYERS], sum(y[k, j] for j=1:N_PLAYERS) == 1)
@constraint(ym, demand[k in 1:N_PLAYERS], sum(y[i, k] for i=1:N_PLAYERS) == 1)

@expression(ym, score, sum(get_score_chess_game(team_host[i], 
                    team_guest[j], k, elo) * xopt[i, k] * y[j, k] 
                    for i=1:N_PLAYERS, j in 1:N_PLAYERS, k in 1:N_PLAYERS))
                                    
@objective(ym, Min, score)
            
solve(ym)
println(getobjectivevalue(ym))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_y)

2.1368277781896134
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  1  0
2     │ 0  1  0  0
3     │ 0  0  0  1
4     │ 1  0  0  0


Y site

In [17]:
nashModel = Model()

@variable(nashModel, y[1:N_PLAYERS, 1:N_PLAYERS], Bin)
@variable(nashModel, p[1:N_PLAYERS])
@variable(nashModel, q[1:N_PLAYERS])
@variable(nashModel, r[1:N_PLAYERS, 1:N_PLAYERS] <= 0) # change

@constraint(nashModel, supply[k in 1:N_PLAYERS], 
                sum(y[k, j] for j=1:N_PLAYERS) == 1)
@constraint(nashModel, demand[k in 1:N_PLAYERS], 
                sum(y[i, k] for i=1:N_PLAYERS) == 1)
                        
@constraint(nashModel, maxcons[i in 1:N_PLAYERS, k in 1:N_PLAYERS],  # change
    p[i] + q[k] + r[i, k] <= 
        sum(get_score_chess_game(team_guest[j], team_host[i], k + 1, elo) 
                                * y[j, k] for j=1:N_PLAYERS))
                                    
@objective(nashModel, Max, sum(p) + sum(q) + sum(r))        

solve(nashModel)
                                    
println(4 - getobjectivevalue(nashModel))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[j, k] for j=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_y)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.137135523262585
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  0  1
2     │ 0  1  0  0
3     │ 0  0  1  0
4     │ 1  0  0  0
p: [0.361051,0.391158,0.454506,0.46859]
q: [-0.038643,0.0944544,-0.0344054,0.166153]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [14]:
nashModel = Model()

@variable(nashModel, y[1:N_PLAYERS, 1:N_PLAYERS], Bin)
@variable(nashModel, p[1:N_PLAYERS])
@variable(nashModel, q[1:N_PLAYERS])
@variable(nashModel, r[1:N_PLAYERS, 1:N_PLAYERS] >= 0) # change

@constraint(nashModel, supply[k in 1:N_PLAYERS], 
                sum(y[k, j] for j=1:N_PLAYERS) == 1)
@constraint(nashModel, demand[k in 1:N_PLAYERS], 
                sum(y[i, k] for i=1:N_PLAYERS) == 1)
                        
@constraint(nashModel, maxcons[i in 1:N_PLAYERS, k in 1:N_PLAYERS],  # change
    p[i] + q[k] + r[i, k] >= 
        sum(get_score_chess_game(team_host[i], team_guest[j], k, elo) 
                                * y[j, k] for j=1:N_PLAYERS))
                                    
@objective(nashModel, Min, sum(p) + sum(q) + sum(r))        

solve(nashModel)
                                    
println(getobjectivevalue(nashModel))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[j, k] for j=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_y)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.137135523262585
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  1  0  0
2     │ 0  0  0  1
3     │ 1  0  0  0
4     │ 0  0  1  0
p: [0.544495,0.514387,0.451004,0.436955]
q: [0.128896,-0.0716988,0.133097,0.0]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [15]:
ym = Model()

@variable(ym, x[1:N_PLAYERS, 1:N_PLAYERS], Bin)

@constraint(ym, supply[k in 1:N_PLAYERS], sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(ym, demand[k in 1:N_PLAYERS], sum(x[i, k] for i=1:N_PLAYERS) == 1)

@expression(ym, score, sum(get_score_chess_game(team_host[i], 
                    team_guest[j], k, elo) * yopt[j, k] * x[i, k] 
                    for i=1:N_PLAYERS, j in 1:N_PLAYERS, k in 1:N_PLAYERS))
                                    
@objective(ym, Max, score)
            
solve(ym)
println(getobjectivevalue(ym))
xopt = getvalue(x)
solution_x = NamedArray(Int[xopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_x)

2.137135523262585
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  0  1
2     │ 0  1  0  0
3     │ 1  0  0  0
4     │ 0  0  1  0


### Prob

The problems: only permutations are taken into account. 


In [25]:
m = Model()

@variable(m, 0 <= x[1:n, 1:n]d <= 1)
@variable(m, z[1:n, 1:n], Bin)

@variable(m, p[1:n])
@variable(m, q[1:n])
@variable(m, r[1:n, 1:n] <= 0)

@constraint(m, imply[i in 1:n, j in 1:n], x[i, j] <= z[i, j])
@constraint(m, zeros[k in 1:n], sum(z[i, k] for i=1:n) <= n - 1)

@constraint(m, demand[k in 1:n], sum(x[i, k] for i=1:n) == 1)
@constraint(m, supply[k in 1:n], sum(x[k, j] for j=1:n) == 1)
            
@constraint(m, mincons[j in 1:n, k in 1:n], 
    p[j] + q[k] + r[j, k] - sum(f(i, j, k) * x[i, k] for i=1:n) <= 0)
@objective(m, Max, sum(p) + sum(q) + sum(r))        

solve(m)
println(getobjectivevalue(m))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:n, k=1:n])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.6814989464276873
5×5 Named Array{Float64,2}
A ╲ B │        1         2         3         4         5
──────┼─────────────────────────────────────────────────
1     │      0.0       0.5       0.0       0.5       0.0
2     │      0.0       0.5       0.0       0.5       0.0
3     │ 0.333333       0.0  0.333333       0.0  0.333333
4     │ 0.333333       0.0  0.333333       0.0  0.333333
5     │ 0.333333       0.0  0.333333       0.0  0.333333
p: [0.457686,0.529441,0.560804,0.565048,0.567871]
q: [0.000216323,1.23691e-12,0.000216323,1.23169e-12,0.000216323]
r: [0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]


In [24]:
nashModel = Model()

@variable(nashModel, 1 >= x[1:N_PLAYERS, 1:N_PLAYERS] >= 0)
@variable(nashModel, p[1:N_PLAYERS])
@variable(nashModel, q[1:N_PLAYERS])
@variable(nashModel, r[1:N_PLAYERS, 1:N_PLAYERS] <= 0)

@constraint(nashModel, supply[k in 1:N_PLAYERS], 
                sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(nashModel, demand[k in 1:N_PLAYERS], 
                sum(x[i, k] for i=1:N_PLAYERS) == 1)
                        
@constraint(nashModel, mincons[j in 1:N_PLAYERS, k in 1:N_PLAYERS], 
    p[j] + q[k] + r[j, k] <= 
        sum(get_score_chess_game(team_host[i], team_guest[j], k, elo) 
                                * x[i, k] for i=1:N_PLAYERS))
                                    
@objective(nashModel, Max, sum(p) + sum(q) + sum(r))        

solve(nashModel)
                                    
println(getobjectivevalue(nashModel))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.1370614481524934
4×4 Named Array{Float64,2}
A ╲ B │   1    2    3    4
──────┼───────────────────
1     │ 0.0  0.5  0.0  0.5
2     │ 0.0  0.5  0.0  0.5
3     │ 0.5  0.0  0.5  0.0
4     │ 0.5  0.0  0.5  0.0
p: [0.457686,0.529441,0.560804,0.565024]
q: [0.012053,0.0,0.012053,0.0]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


In [22]:
nashModel = Model()

@variable(nashModel, 1 >= x[1:N_PLAYERS, 1:N_PLAYERS] >= 0)
@variable(nashModel, p[1:N_PLAYERS])
@variable(nashModel, q[1:N_PLAYERS])
@variable(nashModel, r[1:N_PLAYERS, 1:N_PLAYERS] <= 0)

@constraint(nashModel, supply[k in 1:N_PLAYERS], 
                sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(nashModel, demand[k in 1:N_PLAYERS], 
                sum(x[i, k] for i=1:N_PLAYERS) == 1)
                        
@constraint(nashModel, mincons[j in 1:N_PLAYERS, k in 1:N_PLAYERS], 
    p[j] + q[k] + r[j, k] <= 
        sum(get_score_chess_game(team_host[i], team_guest[j], k, elo) 
                                * x[i, k] for i=1:N_PLAYERS))
                                    
@objective(nashModel, Max, sum(p) + sum(q) + sum(r))        

solve(nashModel)
                                    
println(getobjectivevalue(nashModel))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.681498957689749
5×5 Named Array{Float64,2}
A ╲ B │            1             2             3             4             5
──────┼─────────────────────────────────────────────────────────────────────
1     │          0.0           0.5           0.0           0.5           0.0
2     │          0.0           0.5           0.0           0.5           0.0
3     │     0.293403           0.0      0.413194           0.0      0.293403
4     │     0.400613           0.0      0.198775           0.0      0.400613
5     │     0.305984  -1.59336e-14      0.388032   1.58368e-14      0.305984
p: [0.457686,0.529441,0.560804,0.565048,0.567871]
q: [0.000432495,0.0,-0.000216023,-1.92196e-14,0.000432495]
r: [0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]


In [22]:
xopt[3,1] * xopt[1, 2]*xopt[4, 3]*xopt[2, 4]*xopt[5,5]

0.00925925926671702

In [20]:
2.6814989842282766 - 2.6814989576897488

2.6538527819752744e-8

## Selection from the pool

In [18]:
price2 = [1000, 900, 580, 570, 560] 
elo2 = [2838, 2822, 2750, 2747, 2745]
;

In [72]:
function foo2(i::Int, j::Int, k::Int)
    return get_score_chess_game(elo2[i], elo2[j], k % 2)
end

foo2 (generic function with 1 method)

Question: 
x: choose 2 from 5   
y: choose 2 from the rest 3   (dural p <= 0)
budget

In [74]:
n = 5
n_chosen = 2
inf = -1000
up = 1000
# using Gurobi

# m = Model(solver = GurobiSolver((OutputFlag=0)))
m = Model()

@variable(m, x[1:n, 1:n_chosen], Bin)
@variable(m, p[1:n] <= 0) #dual corresponding to <= 1
@variable(m, q[1:n_chosen])
@variable(m, r[1:n, 1:n_chosen] <= 0)
# @variable(m, s[1:n, 1:n_chosen] >= 0)
@variable(m, t[1:n] <= 0)

@constraint(m, supply[i in 1:n], sum(x[i, k] for k=1:n_chosen) <= 1)
@constraint(m, demand[k in 1:n_chosen], sum(x[i, k] for i=1:n) == 1)
                        
@constraint(m, icons[j in 1:n], t[j] >= (1 - sum(x[j, k] for k=1:n_chosen)) * inf)
@constraint(m, ucons[j in 1:n], t[j] <= p[j] + sum(x[j, k] for k=1:n_chosen))
# @constraint(m, t <= sum(p[j] * (1 - sum(x[j, k] for k=1:n_chosen)) for j=1:n))
                        
@constraint(m, mincons[j in 1:n, k in 1:n_chosen], 
    p[j] + q[k] + r[j, k]  <= sum(foo2(i, j, k) * x[i, k] for i=1:n))
                                                    
@objective(m, Max, sum(t) + sum(q) + sum(r))        

# println(m)
solve(m)
println(getobjectivevalue(m))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:n, k=1:n_chosen])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

1.2289041195972994
5×2 Named Array{Float64,2}
A ╲ B │   1    2
──────┼─────────
1     │ 0.0  1.0
2     │ 1.0  0.0
3     │ 0.0  0.0
4     │ 0.0  0.0
5     │ 0.0  0.0
p: [-1.0,-1.0,-0.00392222,0.0,0.0]
q: [0.653217,0.579609]
r: [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]


In [75]:
ym = Model()

@variable(ym, y[1:n, 1:n_chosen], Bin)

@constraint(ym, supply[j in 1:n], sum(y[j, k] for k=1:n_chosen) <= 1 - sum(xopt[j, k] for k=1:n_chosen))
@constraint(ym, demand[k in 1:n_chosen], sum(y[i, k] for i=1:n) == 1)
                        
@objective(ym, Min, sum(foo2(i, j, k) * xopt[i, k] * y[j, k] for i=1:n, j=1:n, k=1:n_chosen))        

# println(ym)
solve(ym)
println(getobjectivevalue(ym))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[i, k] for i=1:n, k=1:n_chosen])
println(solution_y)

1.2289041195952994
5×2 Named Array{Int64,2}
A ╲ B │ 1  2
──────┼─────
1     │ 0  0
2     │ 0  0
3     │ 0  1
4     │ 1  0
5     │ 0  0


## Trade off

In [81]:
price2 = [1, 0.9, 0.58, 0.57, 0.56] 
elo2 = [2838, 2822, 2750, 2747, 2745]
;

In [77]:
function budget(i::Int) 
    return price2[i]
end

budget (generic function with 1 method)

In [82]:
n = 5
n_chosen = 2
inf = -1000
up = 1000
lambda = 0.5
# using Gurobi

# m = Model(solver = GurobiSolver((OutputFlag=0)))
m = Model()

@variable(m, x[1:n, 1:n_chosen], Bin)
@variable(m, p[1:n] <= 0) #dual corresponding to <= 1
@variable(m, q[1:n_chosen])
@variable(m, r[1:n, 1:n_chosen] <= 0)
# @variable(m, s[1:n, 1:n_chosen] >= 0)
@variable(m, t[1:n] <= 0)

@constraint(m, supply[i in 1:n], sum(x[i, k] for k=1:n_chosen) <= 1)
@constraint(m, demand[k in 1:n_chosen], sum(x[i, k] for i=1:n) == 1)
                        
@constraint(m, icons[j in 1:n], t[j] >= (1 - sum(x[j, k] for k=1:n_chosen)) * inf)
@constraint(m, ucons[j in 1:n], t[j] <= p[j] + sum(x[j, k] for k=1:n_chosen))
# @constraint(m, t <= sum(p[j] * (1 - sum(x[j, k] for k=1:n_chosen)) for j=1:n))
                        
@constraint(m, mincons[j in 1:n, k in 1:n_chosen], 
    p[j] + q[k] + r[j, k]  <= sum((foo2(i, j, k) - lambda * budget(i))* x[i, k] for i=1:n))
                                                    
@objective(m, Max, sum(t) + sum(q) + sum(r))        

# println(m)
solve(m)
println(getobjectivevalue(m))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:n, k=1:n_chosen])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

0.27890411959529937
5×2 Named Array{Float64,2}
A ╲ B │   1    2
──────┼─────────
1     │ 0.0  1.0
2     │ 1.0  0.0
3     │ 0.0  0.0
4     │ 0.0  0.0
5     │ 0.0  0.0
p: [-1.0,-1.0,-0.00392222,0.0,0.0]
q: [0.203217,0.0796092]
r: [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]


In [83]:
ym = Model()

@variable(ym, y[1:n, 1:n_chosen], Bin)

@constraint(ym, supply[j in 1:n], sum(y[j, k] for k=1:n_chosen) <= 1 - sum(xopt[j, k] for k=1:n_chosen))
@constraint(ym, demand[k in 1:n_chosen], sum(y[i, k] for i=1:n) == 1)
                        
@objective(ym, Min, sum((foo2(i, j, k) - lambda * budget(i)) * xopt[i, k] * y[j, k] for i=1:n, j=1:n, k=1:n_chosen))        

# println(ym)
solve(ym)
println(getobjectivevalue(ym))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[i, k] for i=1:n, k=1:n_chosen])
println(solution_y)

0.27890411959529937
5×2 Named Array{Int64,2}
A ╲ B │ 1  2
──────┼─────
1     │ 0  0
2     │ 0  0
3     │ 0  1
4     │ 1  0
5     │ 0  0
